In [ ]:
# Modules

import numpy as np
import scipy.integrate as spint
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import argparse

from dataclasses import dataclass
plt.rcParams['mathtext.fontset'] = 'stix'     


In [ ]:
# Class for the CANN model

class cann_model:
    # define the range of perferred stimuli
    z_min = - np.pi;              
    z_range = 2.0 * np.pi;
    # define the time scale
    tau = 2.0
        
    # function for periodic boundary condition
    def dist(self, c):
        tmp = np.remainder(c, self.z_range)
        
        # routine for numbers
        if isinstance(tmp, (int, float)):
            if tmp > (0.5 * self.z_range):
                return (tmp - self.z_range);
            return tmp;
        
        # routine for numpy arraies
        for tmp_1 in np.nditer(tmp, op_flags=['readwrite']):
            if tmp_1 > (0.5 * self.z_range):
                tmp_1[...] = tmp_1 - self.z_range;
        
        return tmp;
    
    # constructor (?)
    def __init__(self, argument):
        self.k = argument.k;              # rescaled inhibition
        self.beta = argument.beta;        # rescaled STD
        self.beta_f = argument.beta_f;    # flat STD ?
        self.case = argument.case
        self.taud = argument.taud;        # rescaled STD timescale
        self.a = argument.a;              # range of excitatory connection
        self.b = argument.b;              # range of beta modulation
        self.N = argument.N;              # number of units / neurons
        self.dx = self.z_range / self.N     # separation between neurons

        self.rng = np.random.default_rng()
        
        # define perferred stimuli for each neuron
        self.x = (np.arange(0,self.N,1)+0.5) * self.dx + self.z_min;
        
        # difference of x's
        self.x_diff = np.array(
                       [[self.dist(self.x[i] - self.x[j]) 
                         for j in range(self.x.shape[0])]
                        for i in range(self.x.shape[0])]
                       )
        
        # calculate the excitatory couple for each pair of neurons
        self.Jxx = np.exp(-0.5 * np.square(
                      self.x_diff / self.a
                   ) ) / (np.sqrt(2*np.pi) * self.a);
                
        self.y = np.zeros((self.N + self.N * self.N));   # initialize dynamical variables
        self.y[self.N:] = 1.0
        
        
        
        if self.beta_f:
            self.beta = argument.beta
        else:
            self.beta = np.zeros_like(self.x_diff)
            
            for i in np.arange(self.Jxx.shape[1]):
                if self.case == 1:
                    gamma_randm = self.rng.gamma(shape=3.354520641938138, scale=9.743699331037247, size=self.Jxx.shape[0])
                elif self.case == 0:
                    gamma_randm = self.rng.gamma(shape=1.377771974410986, scale=29.196273404252505, size=self.Jxx.shape[0])
                elif self.case == 2:
                    gamma_randm = self.rng.gamma(shape=1, scale=40, size=self.Jxx.shape[0])
                    
                J_sort_idx = np.argsort(self.Jxx[:,i])
                gamma_sort_idx = np.argsort(gamma_randm)
                
                for j in np.arange(self.Jxx.shape[0]):
                    self.beta[J_sort_idx[j],i] = gamma_randm[gamma_sort_idx[j]]               
            
            
#            self.beta = 1.0 / (1.0 + np.exp(-np.square(self.x_diff/self.b)))
            self.beta *= argument.beta / np.mean(self.beta)
        
        
        # Here the first N y elements for u(x,t)
        # and the last N*N elements for p(x,x',t)
        
        self.r = np.zeros((self.N));                     # initialize neuronal activities
        self.input = np.zeros((self.N));                 # initialial the external input
    
    # function for setting external iput for each neuron
    def set_input(self, A, z0):
        self.input = \
        A * np.exp(-0.25 * np.square(self.dist(self.x - z0) / self.a));
    
    # function for calculation of neuronal activity of each neuron
    def cal_r_or_u(self, u_in):
        
        if (u_in.shape[0] > self.N):
            u = u_in[:self.N]
        else:
            u = u_in
            
        u0 = 0.5 * (u + np.abs(u));
        r = np.square(u0);
        B = 1.0 + 0.125 * self.k * np.sum(r) * self.dx \
        / (np.sqrt(2*np.pi) * self.a);
        r = r / B;
        
        return r;
    
    # Calculate the centre of mass of u(x,t)
    def cm_of_u(self):
        u = self.y[:self.N]
        max_i = u.argmax()
        cm = np.dot(self.dist(self.x - self.x[max_i]), u) / u.sum()
        cm = cm + self.x[max_i]
        return cm;
    
    # function for calculation of derivatives
    def get_dydt(self, t, y):
        u = y[:self.N]
        p = y[self.N:].reshape((self.N, self.N))
        r = self.cal_r_or_u(u)
        
        dudt = \
            -u + np.dot(self.Jxx* p , r) * self.dx + self.input;
        dudt = dudt / self.tau;
        
        dpdt = (1-p - self.beta * p * r ) / self.taud
        
        dydt = np.append(dudt, dpdt.reshape(-1))
        
        return dydt

In [ ]:
# Defining a class of input arguments as a workaround 
# for the argparse

@dataclass()
class argument_c:
    k:float
    beta:float
    beta_f:bool
    case:int
    taud:float
    a:float
    b:float
    N:int
    
    def _init__(self, N, k, beta, beta_f, case, taud, a, b):
        self.N = N
        self.k = k
        self.beta = beta
        self.case = case
        self.beta_f = beta_f
        self.taud = taud
        self.a = a
        self.b = b

In [ ]:
np.random.seed(100)

the_beta = 0.001

arg1 = argument_c(N=128, k=0.5, beta=the_beta, beta_f=False, case=0, taud=100, a=0.5, b=0.1)
the_model1 = cann_model(arg1)

arg2 = argument_c(N=128, k=0.5, beta=the_beta, beta_f=False, case=1, taud=100, a=0.5, b=100)
the_model2 = cann_model(arg2)

arg3 = argument_c(N=128, k=0.5, beta=the_beta, beta_f=False, case=2, taud=100, a=0.5, b=100)
the_model3 = cann_model(arg3)

In [ ]:
# Initialize the system

the_model1.set_input(0.5, 0)
the_model2.set_input(0.5, 0)
the_model3.set_input(0.5, 0)
out1 = spint.solve_ivp(the_model1.get_dydt, (0, 1000), the_model1.y, method="RK45");
out2 = spint.solve_ivp(the_model2.get_dydt, (0, 1000), the_model2.y, method="RK45");
out3 = spint.solve_ivp(the_model3.get_dydt, (0, 1000), the_model3.y, method="RK45");

# update the network state in the CANN object
the_model1.y = out1.y[:,-1]
the_model2.y = out2.y[:,-1]
the_model3.y = out3.y[:,-1]

In [ ]:

time_step = 2

# run the simulation and take snapshots every 10 taus
for t in range(0,100,time_step):
    # decide the period of this step
    t0 = t
    t1 = t + time_step
    # run the simulation and update the state in the CANN object
    out1 = spint.solve_ivp(the_model1.get_dydt, (t0, t1), the_model1.y, method="RK45")
    the_model1.y = out1.y[:,-1]
    out2 = spint.solve_ivp(the_model2.get_dydt, (t0, t1), the_model2.y, method="RK45")
    the_model2.y = out2.y[:,-1]
    out3 = spint.solve_ivp(the_model3.get_dydt, (t0, t1), the_model3.y, method="RK45")
    the_model3.y = out3.y[:,-1]
    

the_model1.set_input(0, 1.5)
the_model2.set_input(0, 1.5)
the_model3.set_input(0, 1.5)

In [ ]:
np.mean(the_model1.beta)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15/2.525, 5.75/2.54))

hfont = {"fontname": "Arial"}

# Create the plots
plot1 = ax[0].pcolormesh(the_model1.x, the_model1.x, the_model1.y[the_model1.N:].reshape((the_model1.N, the_model1.N)), vmin=0.94, vmax=1)
ax[0].set_xlabel(r"$x\prime$", fontsize=14)
ax[0].text(-0.4,0.5,r"$x$", fontsize=14, rotation=90, transform=ax[0].transAxes)
ax[0].set_title("Control", **hfont)

plot2 = ax[1].pcolormesh(the_model2.x, the_model2.x, the_model2.y[the_model2.N:].reshape((the_model2.N, the_model2.N)), vmin=0.94, vmax=1)
ax[1].set_xlabel(r"$x\prime$", fontsize=14)
ax[1].set_title("Variation\nNarrowing", **hfont)

plot3 = ax[2].pcolormesh(the_model3.x, the_model3.x, the_model3.y[the_model3.N:].reshape((the_model3.N, the_model3.N)), vmin=0.94, vmax=1)
ax[2].set_xlabel(r"$x\prime$", fontsize=14)
ax[2].set_title("Variation\nWidening", **hfont)

# Add a single colorbar

labels = ["(B)", "(C)", "(D)"]

for axt, lt in zip(ax, labels):
    axt.text(-0.2,1.1,lt, fontsize=14, weight="bold", transform=axt.transAxes, **hfont)

cbar_ax = fig.add_axes([0.9, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
fig.colorbar(plot1, cax=cbar_ax)

plt.subplots_adjust(top=0.8, bottom=0.25, left=0.1, right=0.875, wspace=0.3)

plt.savefig("Figure_2BC.pdf")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15/2.54, 5/2.54))

hfont = {"fontname": "Arial"}

ax1 = ax

ax1.plot(the_model1.x, the_model1.beta[:,64], "k-", lw = 2, label="Control")
#ax1.plot(the_model2.x, the_model2.beta[:,64], "b-.", lw = 2, label="Astrocytic NMDAR Blocked")
ax1.plot(the_model2.x, the_model2.beta[:,64], "r--", lw = 2, label="Variation Narrowing")
ax1.plot(the_model3.x, the_model3.beta[:,64], "b-.", lw = 2, label="Variation Widening")
ax1.plot(the_model2.x, np.mean(the_model3.beta[:,64]) * np.ones_like(the_model2.x), "g-", lw = 2, label="Evenly Distributed")
ax1.set_xlabel(r"$x-x\prime$", fontsize=14)
ax1.set_ylabel(r"$\beta (x,x\prime)$", fontsize=14)
ax1.legend(loc="upper left", handlelength=4, bbox_to_anchor=(1.01,0.99))
ax1.text(-0.2,1.0,r"(A)", transform=ax1.transAxes, weight="bold", fontsize=14, **hfont)

plt.subplots_adjust(left=0.13, right=0.6, top=0.92, bottom=0.25)

plt.savefig("Figure_2A.pdf")

In [ ]:
the_model1.beta[50,:].mean()

In [ ]:
the_model2.beta[50,:].mean()

In [ ]:
the_model2.beta

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(15/2.54, 12/2.54))

axs[0, 1].remove()

axs[0, 0].remove()

ax_main = fig.add_subplot(2, 1, 1)

ax_main.plot(the_model1.x, the_model1.beta[:,64], "r--", lw = 2, label="Control")
ax_main.plot(the_model2.x, the_model2.beta[:,64], "b-.", lw = 2, label="Release Probability\nVariation Narrowing")
ax_main.plot(the_model2.x, 0.001 * np.ones_like(the_model2.x), lw = 2, label="Evenly Distributed")
ax_main.set_xlabel(r"$x-x\prime$", fontsize=14)
ax_main.set_ylabel(r"$\beta (x,x\prime)$", fontsize=14)
ax_main.legend(loc="upper right", bbox_to_anchor=(0.999, 0.99))
ax_main.text(-0.1, 1.05, "(A)", fontsize=14, weight="bold",transform=ax_main.transAxes)

# axs[0,0].plot(the_model1.x, the_model1.beta[:,64], "r--", lw = 2, label="Control")
# axs[0,0].plot(the_model2.x, the_model2.beta[:,64], "b-.", lw = 2, label="Astrocytic NMDAR Blocked")
# axs[0,0].plot(the_model2.x, 0.001 * np.ones_like(the_model2.x), lw = 2, label="Evenly Distributed")
# axs[0,0].set_xlabel(r"$x-x\prime$", fontsize=14)
# axs[0,0].set_ylabel(r"$\beta (x,x\prime)$", fontsize=14)
# axs[0,0].legend(loc="upper right", bbox_to_anchor=(1.2, 0.99))
# axs[0,0].set_title(r"(A)", loc="left")


plot2 = axs[1,0].pcolormesh(the_model1.x, the_model1.x, the_model1.y[the_model1.N:].reshape((the_model1.N, the_model1.N)), vmin=0.94, vmax=1)
axs[1,0].set_xlabel(r"$x\prime$", fontsize=14)
axs[1,0].set_ylabel(r"$x$", fontsize=14)
plt.colorbar(plot2, shrink=1)
axs[1,0].set_title('Control', loc="center")
axs[1,0].text(-0.3, 1.1, "(B)", fontsize=14, weight="bold",transform=axs[1,0].transAxes)

plot3 = axs[1,1].pcolormesh(the_model2.x, the_model2.x, the_model2.y[the_model2.N:].reshape((the_model2.N, the_model2.N)), vmin=0.94, vmax=1)
axs[1,1].set_xlabel(r"$x\prime$", fontsize=14)
axs[1,1].set_ylabel(r"$x$", fontsize=14)
plt.colorbar(plot3, shrink=1)
axs[1,1].set_title('Release Probability\nVariation Narrowing', loc="center")
axs[1,1].text(-0.3, 1.1, "(C)", fontsize=14, weight="bold",transform=axs[1,1].transAxes)

plt.subplots_adjust(left=0.13, right=0.95, top=0.94, bottom=0.1, hspace=0.65, wspace=0.5)

# plt.savefig("Figure_02.pdf")